In [ ]:
import os, numpy as np, pandas as pd, cartopy.crs as ccrs, bokeh
import holoviews as hv, geoviews as gv, datashader as ds
from holoviews.operation.datashader import rasterize, datashade
from colorcet import bmy, bgyw, isolum
from holoviews.util import Dynamic
hv.extension('bokeh', width=100)
from bokeh.models import HoverTool, CustomJSHover

df = pd.read_hdf('./data/rgi62_era5_itmix_df.h5', 'df')
df['latdeg'] = df.CenLat
df = df.rename(columns={'Area':'area'})
df['vol_asl_m3'] = df.vol_itmix_m3 - df.vol_bsl_itmix_m3
df.tail()

In [ ]:
data = gv.Points(df, [('CenLon', 'Longitude'), ('CenLat', 'Latitude')],
                     [('era5_avg_pcp', 'Annual Precipitation (mm/yr)'),
                      ('area', 'Area'),  ('latdeg', 'Latitude (deg)'),
                      ('era5_avg_temp_at_zmed', 'Annual Temperature at avg. altitude (°C)'), 
                      ('Zmed', 'Mean elevation of the glacier'), ('vol_asl_m3', 'Volume asl'),
                      ('vol_bsl_itmix_m3', 'Volume bsl'),
                      ('era5_trend', 'Temperature trend of the last 40 years (°C)')])
total_area = df.area.sum()
#print(data, len(data), total_area)

data = gv.Dataset(gv.operation.project_points(data))

In [ ]:
def slr_calculater(ice_vol):
    """ice_vol in m³ gives slr in mm"""
    rho = 0.9 * 10**3 # kg/m³
    # 1 kg entspricht 1 dm³ --> * 1000 in m³
    A_oc = 362.5 * 10**6 * 10**6 
    slr_m = ice_vol * rho * 1000 / A_oc
    slr_mm = slr_m / 1000
    return slr_mm

In [ ]:
geo_kw    = dict(aggregator=ds.sum('area'), x_sampling=1000, y_sampling=1000)
elev_kw   = dict(cmap='#7d3c98')
temp_kw   = dict(num_bins=50, adjoin=False, normed=False, bin_range=data.range('era5_avg_temp_at_zmed'))
prcp_kw   = dict(num_bins=50, adjoin=False, normed=False, bin_range=(0, 6000))
tren_kw   = dict(num_bins=50, adjoin=False, normed=False, bin_range=data.range('era5_trend'))

size_opts_map = dict(height=450, width=715)
size_opts     = dict(height=200, width=350)
size_opts_bar = dict(height=45,  width=250)
size_opts_slr = dict(height=300, width=120)

geo_opts  = dict(size_opts_map, cmap=bmy, global_extent=False, logz=True, colorbar=True, colorbar_opts={'title':'Area (km²)'}, toolbar='above')
elev_opts = dict(size_opts,     show_grid=True)
temp_opts = dict(size_opts,     fill_color='#f1948a', default_tools=[], toolbar=None, alpha=1.0, ylabel='')
prcp_opts = dict(size_opts,     fill_color='#85c1e9', default_tools=[], toolbar=None, alpha=1.0, ylabel='')
tren_opts = dict(size_opts,     fill_color='#f4d34e', default_tools=[], toolbar=None, alpha=0.85)
slr_opts  = dict(size_opts_slr,  color='orange',      default_tools=[], toolbar=None, xlabel='', show_legend=False, ylabel='Volume in mm sea-level equivalent', yticks=[0, 50, 100, 150,200,250,300,350,390])
glno_opts = dict(size_opts_bar,  color='#326a86',     default_tools=[], toolbar=None, alpha=0.8,  invert_axes=True, show_legend=False, xaxis=None, yaxis=None,)
area_opts = dict(size_opts_bar,  color='#326a86',     default_tools=[], toolbar=None, alpha=0.8,  invert_axes=True, show_legend=False, xaxis=None, yaxis=None,)

In [ ]:
gl_number = len(df)

def geo(data):    return gv.Points(data, crs=ccrs.GOOGLE_MERCATOR).options(alpha=1)
def elev(data):   return data.to(hv.Scatter, 'Zmed', 'latdeg', [])
def temp(data):   return data.hist('era5_avg_temp_at_zmed', **temp_kw).options(**temp_opts)
def prcp(data):   return data.hist('era5_avg_pcp',          **prcp_kw).options(**prcp_opts)
def tren(data):   return data.hist('era5_trend',            **tren_kw).options(**tren_opts)
def slr(data):    return hv.Bars([('asl', slr_calculater(sum(data['vol_asl_m3']))), ('bsl', slr_calculater(sum(data['vol_bsl_itmix_m3'])))]).opts(**slr_opts)
def gl_no(data):  return hv.Bars(('', len(data))                     ).opts(**glno_opts)
def areaa(data):  return hv.Bars(('', sum(data['area']))             ).opts(**area_opts)
def count1(data): return hv.Div('<p style="font-size:15px">Glaciers selected: \n {} of {}'.format(len(data), gl_number)).options(height=20)
def count2(data): return hv.Div('<p style="font-size:15px">Area: {:.0f} km² ({:.1f}%)</font>'.format(np.sum(data['area']), np.sum(data['area']) / total_area * 100)).options(height=20)

static_geo  = rasterize(geo(data),   **geo_kw).options(alpha=0.1, tools=['hover', 'box_select'], active_tools=['box_select'], **geo_opts)
static_elev = datashade(elev(data), **elev_kw).options(alpha=0.1, tools=[         'box_select'], active_tools=['box_select'], **elev_opts)
static_temp = temp(data).options(alpha=0.1)
static_prcp = prcp(data).options(alpha=0.1)
static_tren = tren(data).options(alpha=0.1) 
static_slr  = slr(data).options(alpha=0.1)
static_gl_no= gl_no(data).options(alpha=0.1)
static_area = areaa(data).options(alpha=0.1)


In [ ]:
def combine_selections(**kwargs):
    """
    Combines selections on all available plots into a single selection by index.
    """
    if all(not v for v in kwargs.values()):
        return slice(None)
    selection = {}
    for key, bounds in kwargs.items():
        if bounds is None:
            continue
        elif len(bounds) == 2:
            selection[key] = bounds
        else:
            xbound, ybound = key.split('__')
            selection[xbound] = bounds[0], bounds[2]
            selection[ybound] = bounds[1], bounds[3]
    return sorted(set(data.select(**selection).data.index))

def select_data(**kwargs):
    return data.iloc[combine_selections(**kwargs)] if kwargs else data

from holoviews.streams import Stream, BoundsXY, BoundsX

geo_bounds  = BoundsXY(source=static_geo,  rename= {'bounds':  'CenLon__CenLat'})
elev_bounds = BoundsXY(source=static_elev, rename= {'bounds':  'Zmed__latdeg'})
temp_bounds = BoundsX( source=static_temp, rename= {'boundsx': 'era5_avg_temp_at_zmed'})
prcp_bounds = BoundsX( source=static_prcp, rename= {'boundsx': 'era5_avg_pcp'})
tren_bounds = BoundsX( source=static_tren, rename= {'boundsx': 'era5_trend'})
slr_bounds  = BoundsX( source= static_slr, rename= {'boundsx': 'vol_asl_m3'})
area_bounds = BoundsX( source= static_area, rename={'boundsx': 'area'})


selections  = [geo_bounds, elev_bounds, temp_bounds, prcp_bounds, slr_bounds, area_bounds, tren_bounds]


dyn_data  = hv.DynamicMap(select_data, streams=selections)

dyn_geo   = rasterize(dyn_data.apply(geo),  **geo_kw).options( **geo_opts)
dyn_elev  = datashade(dyn_data.apply(elev), **elev_kw).options(**elev_opts)
dyn_temp  =           dyn_data.apply(temp)
dyn_prcp  =           dyn_data.apply(prcp)
dyn_count1=           dyn_data.apply(count1)
dyn_count2=           dyn_data.apply(count2)
dyn_tren  =           dyn_data.apply(tren)
dyn_slr   =           dyn_data.apply(slr)
dyn_gl_no =           dyn_data.apply(gl_no)
dyn_area  =           dyn_data.apply(areaa)

geo_bg = gv.tile_sources.EsriImagery.options(alpha=1.0, bgcolor="white")

geomap = geo_bg * static_geo  * dyn_geo
elevation       = static_elev * dyn_elev
temperature     = static_temp * dyn_temp
precipitation   = static_prcp * dyn_prcp
trends          = static_tren * dyn_tren
sealevelrise    = static_slr  * dyn_slr
gl_num          = static_gl_no * dyn_gl_no
area_bar        = static_area * dyn_area


def clear_selections(arg=None):
    geo_bounds.update(bounds=None)
    elev_bounds.update(bounds=None)
    temp_bounds.update(boundsx=None)
    prcp_bounds.update(boundsx=None)
    slr_bounds.update(boundsx=None)  
    area_bounds.update(boundsx=None)
    tren_bounds.update(boundsx=None)
    Stream.trigger(selections)

import panel as pn
pn.extension()

clear_button = pn.widgets.Button(name='Clear selection', width=170)
clear_button.param.watch(clear_selections, 'clicks');
title       = '<div style="font-size:35px">World glaciers explorer</div>'
instruction = 'Choose your region of interest by clicking and dragging the mouse in the map ("box-select"). That works also in the other figures. Clear your selection to reset with the "Clear selection"-button and have a look at toolbar of the map: you can zoom with the mouse wheel. <br>' + \
              'See the <a href="https://github.com/panel-demos/glaciers">Jupyter notebook</a> source code for how to build apps like this!'
oggm_logo   = '<a href="https://edu.oggm.org"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-explorer/dev/oggm.gif" width=180></a>'
pn_logo     = '<a href="https://panel.pyviz.org"><img src="http://panel.pyviz.org/_static/logo_stacked.png" width=46></a>'
holo_logo   = '<a href="http://holoviz.org/"><img src="https://raw.githubusercontent.com/pyviz/holoviews/master/doc/_static/logo.png" width=46></a>'
dasha_logo   = '<a href="http://datashader.org/"><img src="https://raw.githubusercontent.com/pyviz/datashader/master/doc/_static/datashader-logo.png" width=46></a>'

header = pn.Row(pn.Pane(oggm_logo),  pn.layout.Spacer(width=30), 
                pn.Column(pn.Pane(title, width=400), pn.Pane(instruction, width=500)),
                pn.layout.HSpacer(), pn.Column(pn.layout.Spacer(height=70), ), 
                pn.Pane(pn_logo, width=140))

plots = pn.Row(pn.layout.Spacer(width=25), trends, pn.layout.Spacer(width=5), temperature, pn.layout.Spacer(width=5), precipitation, pn.layout.Spacer(width=5), elevation)
overview = pn.Row(pn.Column(pn.Pane(dyn_count1), pn.layout.Spacer(height=1), gl_num, pn.Pane(dyn_count2), pn.layout.Spacer(height=1), area_bar, pn.layout.Spacer(height=15)), sealevelrise)
logos = pn.Row(pn_logo, pn.layout.Spacer(height=0), holo_logo, pn.layout.Spacer(height=0), dasha_logo)

pn.Column(pn.Row(pn.Column(oggm_logo, pn.layout.Spacer(height=20), logos, pn.layout.Spacer(height=40), clear_button), 
                 pn.layout.Spacer(width=10), 
                 pn.Column(pn.Pane(title, width=400), pn.Pane(instruction, width=470), pn.Row(pn.layout.Spacer(width=10),overview,)), 
                 pn.layout.Spacer(width=45), 
                 geomap),
          
          pn.layout.Spacer(height=5),
          plots).show()